## 1D CNN Model - Benchmarking

In [ ]:
"""
@authors: Maxime Tchibozo, Donggeun Kim
@affiliation: NYSPI, Columbia University
@date: Oct 2018 - Oct 2020
@overview: Comparing the performance of different Machine Learning algorithms on the Experiment A task.
@input: Hyperaligned voxels with labels as numpy arrays (X_hyp_v2.csv and Y_hyp_v2.csv).
@output: Metrics for different classical algorithms.
"""

In [1]:
import numpy as np
import pandas as pd
import keras
from keras import backend as K
from keras.callbacks import History
import pickle
from keras import initializers
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.layers.convolutional import Conv1D, MaxPooling1D, ZeroPadding1D
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import glob
import os
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure, show
from matplotlib.ticker import MaxNLocator
import matplotlib as mpl
from sklearn import preprocessing
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from xgboost import XGBClassifier

from sklearn.metrics import roc_curve,roc_auc_score

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
folder_root = '/content/drive/My Drive/BrainStateClassification/code/'#Update to your main folder, this folder must contain a data and a result subfolder
folder_data = folder_root + 'data/' #folder containing all the .mat files
folder_result = folder_root + 'data/plateau' #folder where your models and logs will be stored


In [4]:
os.chdir(folder_data)

In [5]:
Y = np.loadtxt('Y_hyp_v2.csv',delimiter=",")
X = np.loadtxt('X_hyp_v2.csv',delimiter=",")

In [6]:
from collections import Counter
Counter(Y)

Counter({0.0: 2200, 1.0: 1100, 2.0: 1100})

In [ ]:
np.shape(X)

(4400, 300)

## Comparison with other models

Here we compare the 1DCNN performance with SVM, XGBoost

In [ ]:
rbf_svm_scores = []

tmp = list(range(0,4800,400))
X_Hold_out = [] #Holdout = testing | Max
X_CV=[]
Y_Hold_out = []
Y_CV=[]
for i in range(len(tmp)-1):
    X_Hold_out+=[X[tmp[i]:tmp[i+1]]]
    Y_Hold_out+=[Y[tmp[i]:tmp[i+1]]]
    X_CV += [np.concatenate((X[0:tmp[i]],X[tmp[i+1]:]))]
    Y_CV += [np.concatenate((Y[0:tmp[i]],Y[tmp[i+1]:]))]
N_Leave_One_Subject = len(tmp)-1


for leave_one_idx in range(N_Leave_One_Subject): 
    print(leave_one_idx)
    X_train = X_CV[leave_one_idx] # 300 refers to N_features
    Y_train = Y_CV[leave_one_idx].reshape(-1,1)
    X_test = X_Hold_out[leave_one_idx]
    Y_test = Y_Hold_out[leave_one_idx]

    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train).reshape(-1,300,1)
    X_test = scaler.transform(X_test).reshape(-1,300,1)

    X_train, Y_train = RandomOverSampler(random_state=0).fit_resample(X_train.reshape(-1,300),Y_train.flatten())
    X_test, Y_test = RandomOverSampler(random_state=0).fit_resample(X_test.reshape(-1,300),Y_test.flatten())

    svm_rbf = SVC(kernel='rbf')
    svm_rbf.fit(X_train,Y_train)
    rbf_svm_scores.append(svm_rbf.score(X_test,Y_test))
    print(rbf_svm_scores[-1])




0


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


0.7666666666666667
1


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


0.83
2


...

0.7516666666666667
9


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


0.7666666666666667
10


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


0.8383333333333334


In [ ]:
linear_svm_scores = []


for leave_one_idx in range(N_Leave_One_Subject): 
    print(leave_one_idx)
    X_train = X_CV[leave_one_idx] # 300 refers to N_features
    Y_train = Y_CV[leave_one_idx].reshape(-1,1)
    X_test = X_Hold_out[leave_one_idx]
    Y_test = Y_Hold_out[leave_one_idx]

    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train).reshape(-1,300,1)
    X_test = scaler.transform(X_test).reshape(-1,300,1)

    X_train, Y_train = RandomOverSampler(random_state=0).fit_resample(X_train.reshape(-1,300),Y_train.flatten())
    X_test, Y_test = RandomOverSampler(random_state=0).fit_resample(X_test.reshape(-1,300),Y_test.flatten())

    svm_linear = LinearSVC()
    svm_linear.fit(X_train,Y_train)
    linear_svm_scores.append(svm_linear.score(X_test,Y_test))
    print(linear_svm_scores[-1])

0


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn

0.7633333333333333
1


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


0.7633333333333333
2


...

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


0.7466666666666667
9


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


0.75
10
0.715


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
random_forest_scores = []


for leave_one_idx in range(N_Leave_One_Subject): 
    print(leave_one_idx)
    X_train = X_CV[leave_one_idx] # 300 refers to N_features
    Y_train = Y_CV[leave_one_idx].reshape(-1,1)
    X_test = X_Hold_out[leave_one_idx]
    Y_test = Y_Hold_out[leave_one_idx]

    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train).reshape(-1,300,1)
    X_test = scaler.transform(X_test).reshape(-1,300,1)

    X_train, Y_train = RandomOverSampler(random_state=0).fit_resample(X_train.reshape(-1,300),Y_train.flatten())
    X_test, Y_test = RandomOverSampler(random_state=0).fit_resample(X_test.reshape(-1,300),Y_test.flatten())

    random_forest = RandomForestClassifier()
    random_forest.fit(X_train,Y_train)
    random_forest_scores.append(random_forest.score(X_test,Y_test))
    print(random_forest_scores[-1])

0


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


0.7233333333333334
1


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


0.765
2


...

0.7116666666666667
9


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


0.7283333333333334
10


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


0.82


In [ ]:

xgb_scores = []


for leave_one_idx in range(N_Leave_One_Subject): 
    print(leave_one_idx)
    X_train = X_CV[leave_one_idx] # 300 refers to N_features
    Y_train = Y_CV[leave_one_idx].reshape(-1,1)
    X_test = X_Hold_out[leave_one_idx]
    Y_test = Y_Hold_out[leave_one_idx]

    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train).reshape(-1,300,1)
    X_test = scaler.transform(X_test).reshape(-1,300,1)

    X_train, Y_train = RandomOverSampler(random_state=0).fit_resample(X_train.reshape(-1,300),Y_train.flatten())
    X_test, Y_test = RandomOverSampler(random_state=0).fit_resample(X_test.reshape(-1,300),Y_test.flatten())

    xgb_model = XGBClassifier()
    xgb_model.fit(X_train,Y_train)
    xgb_scores.append(xgb_model.score(X_test,Y_test))
    print(xgb_scores[-1])


0


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


0.7183333333333334
1


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


0.76
2


...

0.7783333333333333
9


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


0.7216666666666667
10


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


0.815


In [ ]:

lda_scores = []


for leave_one_idx in range(N_Leave_One_Subject): 
    print(leave_one_idx)
    X_train = X_CV[leave_one_idx] # 300 refers to N_features
    Y_train = Y_CV[leave_one_idx].reshape(-1,1)
    X_test = X_Hold_out[leave_one_idx]
    Y_test = Y_Hold_out[leave_one_idx]

    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train).reshape(-1,300,1)
    X_test = scaler.transform(X_test).reshape(-1,300,1)

    X_train, Y_train = RandomOverSampler(random_state=0).fit_resample(X_train.reshape(-1,300),Y_train.flatten())
    X_test, Y_test = RandomOverSampler(random_state=0).fit_resample(X_test.reshape(-1,300),Y_test.flatten())

    lda_model = LinearDiscriminantAnalysis()
    lda_model.fit(X_train,Y_train)
    lda_scores.append(lda_model.score(X_test,Y_test))
    print(lda_scores[-1])


0


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


0.7516666666666667
1


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


0.735
2


...

0.7383333333333333
9


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


0.7516666666666667
10


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


0.7416666666666667


In [ ]:
cnn_scores = [0.868,0.853,0.857,0.820,0.747,0.873,0.850,0.868,0.870,0.863,0.870]


In [ ]:
pd.DataFrame({'CNN test acc':cnn_scores,'SVM(Linear) test acc':linear_svm_scores,'SVM(rbf) test acc':rbf_svm_scores,'Random Forest test acc':random_forest_scores,'XGBoost test acc':xgb_scores,'LDA test acc':lda_scores})

,CNN test acc,SVM(Linear) test acc,SVM(rbf) test acc,Random Forest test acc,XGBoost test acc,LDA test acc
0,0.868,0.763333,0.766667,0.723333,0.718333,0.751667
1,0.853,0.763333,0.830000,0.765000,0.760000,0.735000
2,0.857,0.753333,0.776667,0.750000,0.736667,0.735000
3,0.820,0.766667,0.766667,0.696667,0.733333,0.771667
4,0.747,0.715000,0.706667,0.686667,0.700000,0.705000
5,0.873,0.758333,0.821667,0.753333,0.765000,0.778333
6,0.850,0.715000,0.796667,0.756667,0.753333,0.735000
7,0.868,0.773333,0.831667,0.743333,0.743333,0.790000
8,0.870,0.746667,0.751667,0.711667,0.778333,0.738333
9,0.863,0.750000,0.766667,0.728333,0.721667,0.751667


In [ ]:
#**Comparison to the Hanson et al (2004) neural net**
#each subject, we created a 10-hidden node, eight-waymulti-class NN classifier. We used the hyperbolic tangent activa-tion transfer function for its hidden nodes, softmax activation

import keras
def create_model_2():
    model = Sequential()
    model.add(Dense(10, activation='tanh', input_shape= (300,1)))
    model.add(Flatten())
    model.add(Dense(3, activation='softmax')) 
    return model
model = create_model_2()
model.summary()

Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_75 (Dense)             (None, 300, 10)           20        
_________________________________________________________________
flatten_22 (Flatten)         (None, 3000)              0         
_________________________________________________________________
dense_76 (Dense)             (None, 3)                 9003      
Total params: 9,023
Trainable params: 9,023
Non-trainable params: 0
_________________________________________________________________


In [ ]:
NN_2004_acc = []

batch_size = 64

tmp = list(range(0,4800,400))
X_Hold_out = [] #Holdout = testing | Max
X_CV=[]
Y_Hold_out = []
Y_CV=[]
for i in range(len(tmp)-1):
    X_Hold_out+=[X[tmp[i]:tmp[i+1]]]
    Y_Hold_out+=[Y[tmp[i]:tmp[i+1]]]
    X_CV += [np.concatenate((X[0:tmp[i]],X[tmp[i+1]:]))]
    Y_CV += [np.concatenate((Y[0:tmp[i]],Y[tmp[i+1]:]))]
N_Leave_One_Subject = len(tmp)-1


for learn_rate in [0.0045]:
    for leave_one_idx in range(N_Leave_One_Subject): 
        print(leave_one_idx)
        X_train = X_CV[leave_one_idx] # 300 refers to N_features
        Y_train = Y_CV[leave_one_idx].reshape(-1,1)
        X_test = X_Hold_out[leave_one_idx]
        Y_test = Y_Hold_out[leave_one_idx]
        
        scaler = preprocessing.StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train).reshape(-1,300,1)
        X_test = scaler.transform(X_test).reshape(-1,300,1)

#Y_..._onehot contains one hot encoded Y_... i.e, if Y[i] = 0, Y_onehot[i] = [1,0,0] 

        X_train, Y_train = RandomOverSampler(random_state=0).fit_resample(X_train.reshape(-1,300),Y_train.flatten())
        X_train = X_train.reshape(-1,300,1)
        X_test, Y_test = RandomOverSampler(random_state=0).fit_resample(X_test.reshape(-1,300),Y_test.flatten())
        X_test = X_test.reshape(-1,300,1)

        Y_train_onehot = np.eye(3)[Y_train.astype(int)].reshape(-1,3)
        Y_test_onehot = np.eye(3)[Y_test.astype(int)].reshape(-1,3)

        print("to create")
        model = create_model_2()
        model.compile(loss=keras.losses.categorical_crossentropy,
                      optimizer=keras.optimizers.Adam(lr=learn_rate),
                      metrics=['accuracy'])

        batch_size = 32
        epochs = 50 #20 epochs now

        print("Before Fit")
        model.fit(X_train, Y_train_onehot,
                  batch_size=batch_size,
                  epochs=epochs,
                  verbose=1,
                  validation_data=(X_test, Y_test_onehot))
        score, acc = model.evaluate(X_test, Y_test_onehot, batch_size=batch_size,verbose=0)
        print(f'Test Accuracy for learning rate {learn_rate} CV index {leave_one_idx} is :', "{:.3f}".format(acc))
        NN_2004_acc.append(acc)


0
to create
Before Fit
Epoch 1/50


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


188/188 [==============================] - 1s 3ms/step - loss: 0.6891 - accuracy: 0.7300 - val_loss: 0.6072 - val_accuracy: 0.7300
Epoch 2/50
188/188 [==============================] - 0s 3ms/step - loss: 0.5448 - accuracy: 0.7800 - val_loss: 0.6512 - val_accuracy: 0.7300
Epoch 3/50
188/188 [==============================] - 0s 3ms/step - loss: 0.5114 - accuracy: 0.7988 - val_loss: 0.5936 - val_accuracy: 0.7367
Epoch 4/50
188/188 [==============================] - 0s 3ms/step - loss: 0.4798 - accuracy: 0.8072 - val_loss: 0.5965 - val_accuracy: 0.7500
Epoch 5/50
188/188 [==============================] - 0s 3ms/step - loss: 0.4839 - accuracy: 0.8078 - val_loss: 0.5656 - val_accuracy: 0.7500
Epoch 6/50
188/188 [==============================] - 0s 2ms/step - loss: 0.4833 - accuracy: 0.8053 - val_loss: 0.5867 - val_accuracy: 0.7483
Epoch 7/50
188/188 [==============================] - 0s 3ms/step - loss: 0.4596 - accuracy: 0.8127 - val_loss: 0.5981 - val_accuracy: 0.7517
Epoch 8/50
188/18

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


188/188 [==============================] - 1s 4ms/step - loss: 0.7501 - accuracy: 0.7262 - val_loss: 0.5995 - val_accuracy: 0.7417
Epoch 2/50
188/188 [==============================] - 1s 3ms/step - loss: 0.5304 - accuracy: 0.7845 - val_loss: 0.7134 - val_accuracy: 0.7333
Epoch 3/50
188/188 [==============================] - 1s 3ms/step - loss: 0.5239 - accuracy: 0.7950 - val_loss: 0.5246 - val_accuracy: 0.7667
Epoch 4/50
188/188 [==============================] - 1s 3ms/step - loss: 0.4962 - accuracy: 0.8032 - val_loss: 0.6787 - val_accuracy: 0.7250
Epoch 5/50
188/188 [==============================] - 1s 3ms/step - loss: 0.4794 - accuracy: 0.8075 - val_loss: 0.6232 - val_accuracy: 0.7233
Epoch 6/50
188/188 [==============================] - 1s 3ms/step - loss: 0.4485 - accuracy: 0.8203 - val_loss: 0.5876 - val_accuracy: 0.7500
Epoch 7/50
188/188 [==============================] - 1s 3ms/step - loss: 0.4771 - accuracy: 0.8112 - val_loss: 0.5957 - val_accuracy: 0.7433
Epoch 8/50
188/18

...

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


188/188 [==============================] - 1s 3ms/step - loss: 0.6937 - accuracy: 0.7348 - val_loss: 0.6761 - val_accuracy: 0.7033
Epoch 2/50
188/188 [==============================] - 1s 3ms/step - loss: 0.5249 - accuracy: 0.7897 - val_loss: 0.8554 - val_accuracy: 0.6783
Epoch 3/50
188/188 [==============================] - 0s 3ms/step - loss: 0.5087 - accuracy: 0.7957 - val_loss: 0.6517 - val_accuracy: 0.7217
Epoch 4/50
188/188 [==============================] - 0s 3ms/step - loss: 0.4677 - accuracy: 0.8123 - val_loss: 0.6647 - val_accuracy: 0.7200
Epoch 5/50
188/188 [==============================] - 1s 3ms/step - loss: 0.4699 - accuracy: 0.8147 - val_loss: 0.7067 - val_accuracy: 0.7050
Epoch 6/50
188/188 [==============================] - 1s 3ms/step - loss: 0.4454 - accuracy: 0.8195 - val_loss: 0.6901 - val_accuracy: 0.7083
Epoch 7/50
188/188 [==============================] - 1s 3ms/step - loss: 0.4463 - accuracy: 0.8223 - val_loss: 0.8060 - val_accuracy: 0.6617
Epoch 8/50
188/18

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


188/188 [==============================] - 1s 3ms/step - loss: 0.6580 - accuracy: 0.7387 - val_loss: 0.5521 - val_accuracy: 0.7800
Epoch 2/50
188/188 [==============================] - 0s 3ms/step - loss: 0.5387 - accuracy: 0.7850 - val_loss: 0.5548 - val_accuracy: 0.7800
Epoch 3/50
188/188 [==============================] - 0s 3ms/step - loss: 0.5095 - accuracy: 0.7905 - val_loss: 0.5458 - val_accuracy: 0.7417
Epoch 4/50
188/188 [==============================] - 0s 2ms/step - loss: 0.4928 - accuracy: 0.8037 - val_loss: 0.5751 - val_accuracy: 0.7583
Epoch 5/50
188/188 [==============================] - 0s 3ms/step - loss: 0.4726 - accuracy: 0.8087 - val_loss: 0.5724 - val_accuracy: 0.7167
Epoch 6/50
188/188 [==============================] - 0s 3ms/step - loss: 0.4641 - accuracy: 0.8070 - val_loss: 0.5813 - val_accuracy: 0.7550
Epoch 7/50
188/188 [==============================] - 0s 3ms/step - loss: 0.4649 - accuracy: 0.8098 - val_loss: 0.6166 - val_accuracy: 0.7483
Epoch 8/50
188/18

KeyboardInterrupt: ignored

In [ ]:
np.std([0.735, 0.768, 0.740, 0.747, 0.672, 0.742, 0.743, 0.763, 0.778, 0.745, 0.760])

0.02635015329200945